# Prev

## include

In [78]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")
include("../../src/run_event.jl")

RunFusionAggregates (generic function with 1 method)

## model

In [79]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    ) 
)

# Checking temp data for store
check_data("Test.xyz")
check_data("ErrorInit.xyz")

  0.000000 seconds (4 allocations: 144 bytes)


# Run

In [80]:
# # Almost Complete
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.162);

In [81]:
Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.162);

# Run Model
RunFusionAggregates(model::ModelSet, Par1, Par2, 15)

Run One Aggregate... 100%|███████████████████████████████| Time: 0:01:17m
Fusion of Two Aggregates... 100%|████████████████████████| Time: 0:02:16


1×5008 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 14  8  13  9  10  8  14  14  12  10  10  …  9  14  13  10  10  8  10  14  14

1000×5008 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 106   40  145  303   17   55   11  …  5002  4940  4994  4995  4966  4963
  31  222  125   74   69   44   39     5008  4951  4820  4945  4772  4875
 101  510   33   67   60   46   62     4990  4992  5000  4971  4784  5003
 227  224  242   75   11   72   53     5006  4900  4943  4996  4773  4885
 122  214  124   66   13  107   69     4971  4934  4847  5003  4877  4971
 120  223  253   57   68  116   45  …  4996  4950  4880  4910  4968  4903
 228  205  390    9   16  366   54     4888  4998  4838  4954  4785  4965
 238   30   32  171   10  404   60     4963  4941  4765  5002  4878  5001
  50    0  134   76  121    0   13     5001  4988     0  4956  4974  5002
 104    0  115    0   51    0  143        0  4999     0  4963  4879  4977
 215    0   55    0    0    0    8  …     0     0     0     0  4891  4957
 216    0  366    0    0    0   12     4910     0     0     0  4876  4888
 103    0   72    0    0    0   61        0     0     0     

(17, CartesianIndex(1, 339))

# review

## mini test

In [82]:
# J , I = 10 , 5
# Neigbor = true

# check_data("miniTest.xyz")
# open("miniTest.xyz", "a") do f
#     write(f, "$(size(points, 1))\n")
#     write(f, "t=0\n")
#     writedlm(f,hcat(outline,Matrix(points)), ' ')
# end

# for j =1:J
#     for i = 1:I
#         # Compute the forces between each pair of particles in `agg` and their displacement.
#         threads=(16,3)
#         @cuda(
#             threads=threads,
#             blocks=(cld.(size(points,1)+1,threads[1]),1),
#             sum_force!(idx,random,idx_sum,points,force,ParForce,ParCont,dt,1)
#         )

#     end

#     if Neigbor == true
#         # Calculating Distance Matrix
#         threads=(64)
#         @cuda(
#             threads=threads,
#             blocks=cld.(size(points,1),threads),
#             dist_kernel!(idx,idx_cont,idx_sum,dist,points,r_max)
#         )
#     end

#     if j%200 == 1
#         prev_points = copy(points)
#         open("miniTest.xyz", "a") do f
#             write(f, "$(size(points, 1))\n")
#             write(f, "t=$(j*J)\n")
#             writedlm(f,hcat(outline,Matrix(points)), ' ')
#         end
#     end
# end

# val = 50
# println("BIG Val        = $(val)")
# println("Any BIG value? = $(any(points .> val))")
# println("Any Nan Value? = $(any(isnan.(points)))")
# display(points)
# display(force)